In [14]:
#!pip install folium
# !pip install geopandas

In [15]:
import pandas as pd
import folium
from shapely.geometry import Point
import geopandas as gpd

In [16]:
# Charger le fichier CSV et ne garder que la data relative à la Seine Maritime:
df = pd.read_csv(r"data\CSV\charging_station.csv", sep=";")
df=df[df['code_insee_commune'].str.startswith('75')] #partie personnalisable selon le département voulu 'exemple de la seine maritime
df['code_insee_commune'].unique()

array(['75056'], dtype=object)

In [17]:
df.shape

(269, 41)

In [18]:
#show columns with at least one missing value :

df.columns[~(df.isna().any())]

Index(['X', 'Y', 'id_station_itinerance', 'code_insee_commune',
       'coordonneesXY', 'id_pdc_itinerance', 'condition_acces',
       'accessibilite_pmr', 'observations', 'date_maj'],
      dtype='object')

In [19]:
# Créer des objets Point pour chaque ligne
geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]
len(geometry)

269

In [20]:
# Créer un GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.crs='EPSG:4326' #attribute of the class geodataframe
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
# Définir le système de coordonnées (WGS84)
# gdf.set_crs(epsg=4326, inplace=True)

In [22]:
# Initialiser la carte centrée sur la moyenne des coordonnées
map = folium.Map(location=[df['Y'].mean(), df['X'].mean()], zoom_start=12)

# Ajouter des couches de tuiles supplémentaires avec attribution
folium.TileLayer('Stamen Toner', attr='Map data © OpenStreetMap contributors').add_to(map)
folium.TileLayer('Stamen Watercolor', attr='Map data © OpenStreetMap contributors').add_to(map)
folium.TileLayer('CartoDB positron', attr='Map data © OpenStreetMap contributors').add_to(map)
folium.TileLayer('CartoDB dark_matter', attr='Map data © OpenStreetMap contributors').add_to(map)
folium.TileLayer('Stamen Terrain', attr='Map data © OpenStreetMap contributors').add_to(map)

# Ajouter un contrôle de couche
folium.LayerControl().add_to(map)

# Ajouter des points à la carte avec des liens Street View dans les popups
for idx, row in df.iterrows():
    # URL de Google Street View
    street_view_url = f"http://maps.google.com/?cbll={row['Y']},{row['X']}&layer=c"
    
    # Popup avec lien vers Street View
    popup_content = f"{row['observations']}<br><a href='{street_view_url}' target='_blank'>View on Street View</a>"
    
    folium.Marker(
        location=[row['Y'], row['X']],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=row['code_insee_commune']
    ).add_to(map)

In [23]:
# Sauvegarder la carte dans un fichier HTML
map.save('output carte\carte_interactive.html')

In [24]:
# Afficher la carte dans un notebook Jupyter (optionnel)
from IPython.display import display
display(map)